## 使用反卷积技术

In [ ]:
!pip install tensorflow

!pip install matplotlib 
import tensorflow as tf 

from tensorflow.keras.models import Model 

from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, UpSampling2D 

import numpy as np 

import matplotlib.pyplot as plt 

### 问题分析
使用Transpose Convolution 反卷积技术，将fashion mnist数据集中的28*28图像扩展到18*128

### 数据准备

In [ ]:
fmnist = tf.keras.datasets.fashion_mnist
# 载入 Fashion MNIST 数据集，数据是 28x28 的灰度图，共 10 个类别
(training_images, training_labels), (test_images, test_labels) = fmnist.load_data()

# 归1化（normalization），将像素值从 [0, 255] 变为 [0, 1]，有助于模型训练。
training_images=training_images / 255.0
test_images=test_images / 255.0

### 建模训练

In [ ]:
# 输入层
input_layer = Input(shape=(28, 28, 1))

# 第一卷积层
conv_layer = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)

# 上采样1，28x28 -> 56x56
upsample_layer1 = UpSampling2D(size=(2, 2))(conv_layer)

# 第二卷积层
conv_layer_2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(upsample_layer1)

# 上采样2，56x56 -> 112x112
upsample_layer2 = UpSampling2D(size=(2, 2))(conv_layer_2)

# 第三卷积层
conv_layer_3 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(upsample_layer2)

# 上采样3，112x112 -> 128x128
upsample_layer3 = UpSampling2D(size=(2, 2))(conv_layer_3)

# 最后转置卷积层，输出 128x128 图像
transpose_conv_layer = Conv2DTranspose(filters=1, kernel_size=(3, 3), activation='sigmoid', padding='same')(upsample_layer3)

# 定义模型
model = Model(inputs=input_layer, outputs=transpose_conv_layer)

# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error')